# Here in this file, We will try to prepare the dataset

<h3> Lets first observe the default dataset </h3>

In [162]:
import tarfile 
import re
from datetime import datetime
from collections import namedtuple, Counter
import pandas as pd 
import altair as alt
import os

In [61]:
emails = "emails.csv"
df = pd.read_csv(emails)

In [62]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


# Working with cleaning the dataset

In [66]:
tar = tarfile.open("emails.tar.gz", "r")

In [67]:
items = tar.getmembers()

In [68]:
Email = namedtuple('Email', 'Date, From, To, Subject, Cc, Bcc, Message')

In [69]:
# figure out structure of each file
def get_msg(item_number):
    f = tar.extractfile(items [item_number])
    try:
        date = from_ = to = subject = cc = bcc = messages= ""
        in_to = False
        in_message = False
        to = []
        message = []
        item = f. read().decode()
        item = item.replace('\r', '').replace('\t', '')
        lines = item.split( '\n')
        
        # print(Len (Lines))
        for num, line in enumerate(lines):
            
            if line.startswith('Date:') and not date:
                date = datetime.strptime(' '.join(line.split('Date: ')[1].split()[:-2]), '%a, %d %b %Y %H:%M:%S')
            
            elif line.startswith('From:') and not from_:
                from_ = line.replace('From:', '').strip()
                                     
            elif line.startswith('To:') and not to:
                in_to = True
                to = line.replace('To:','').replace(',',''). replace(';', '').split()
                                                
            elif line.startswith('Subject:') and not subject:
                in_to = False
                subject = line.replace('Subject:', '').strip()
                                       
            elif line.startswith('Cc:') and not cc:
                cc = line.replace( 'Cc:','').replace(',', '').replace(';','').split()
                                       
            elif line.startswith('Bcc:') and not bcc:
                bcc = line.replace( 'Bcc:','').replace(',', '').replace(';', "").split()
                                       
            elif in_to:
                to.extend(line.replace(',','').split())                
            elif line.startswith('Subject:') and not subject:
                in_to = False         
            elif line.startswith('X-FileName'):
                in_message = True
            elif in_message:
                message.append(line)
                                       
                                       
        to  = ';'.join(to).strip()
        cc  = ';'.join(cc).strip()
        bcc = ';'.join(bcc).strip()
        message = ';'.join(message).strip ()
        email = Email (date, from_ , to, subject, cc, bcc, message)
        return email
    except Exception as e:
        return e
            

            

In [122]:
## So we have many attributes now.

# 'Email', 'Date, From, To, Subject, Cc, Bcc, Message'


def printer(number):
    msg = get_msg(number)
    print(msg.Date)
    print(msg.From)
    print(msg.To)
    print(msg.Subject)
    print(msg.Cc)
    print(msg.Bcc)
    print(msg.Message)


printer(300)

2001-10-26 09:43:32
ava.garcia@enron.com
darrell.schoolcraft@enron.com;lynn.blair@enron.com
TW Attendee Spreadsheet


;Attached is the spreadsheet of those attending the Transwestern Update Mtg.  names highlighted in BLUE are people who will be attending the TRAINING session.;;Thanks,;Ava;;


In [86]:
len(items)

520901

In [89]:
520901/2


260450.5

In [91]:
260450/2

130225.0

In [93]:
130225/2

65112.5

In [95]:
65112/2

32556.0

In [97]:
32556.0/2


16278.0

In [170]:
# Step-1. Parse through the dataset
def parse_dataset(items):
    emails = []
    for i in range(len(items)):
        email = get_msg(i)
        if isinstance(email, Email) and any(val is not None for val in email):
            emails.append(email)
    return emails

# Step-2. Create a dataframe out of the parsed emails.
def create_dataframe(emails):
    return pd.DataFrame(emails, columns=Email._fields)


# Step-. Split and Save the dataset
def split_and_save(df, counts, output_dir="splitted"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    for i in range(len(counts)):
        df_part = df[0:counts[i]]
        print(0, '-->', counts[i]) ## here is the print function to indicate the split numbers
        df_part.to_csv(os.path.join(output_dir, f"emails_part_{i + 1}.csv"), index=False)
        

## Just call this method inorder for the magic to happen        
def clean_now():
    emails = parse_dataset(items)
    df = create_dataframe(emails)
    counts = [16278, 32556, 65112, 130225, 260450, 520901]
    split_and_save(df, counts)
    

clean_now()

0 --> 16278
0 --> 32556
0 --> 65112
0 --> 130225
0 --> 260450
0 --> 520901


In [171]:
# Lets read the saved files of dataframes now

df1 = pd.read_csv('splitted/emails_part_1.csv')
df2 = pd.read_csv('splitted/emails_part_2.csv')
df3 = pd.read_csv('splitted/emails_part_3.csv')
df4 = pd.read_csv('splitted/emails_part_4.csv')
df5 = pd.read_csv('splitted/emails_part_5.csv')
df6 = pd.read_csv('splitted/emails_part_6.csv')


In [172]:
df.head(45)
df2.shape
#df1.head()

(32556, 7)